<a href="https://colab.research.google.com/github/ilham-bintang/SSD_Simple/blob/main/SSD_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install matplotlib opencv-python

In [21]:
!unzip images.zip

Archive:  images.zip
   creating: images/
   creating: images/test/
  inflating: images/test/2.jpg       
  inflating: images/test/3.png       
  inflating: images/test/1.jpg       


In [3]:
!python -m pip install -U matplotlib

     |████████████████████████████████| 11.6MB 6.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
!apt-get install tk-dev libpng-dev libffi-dev dvipng texlive-latex-base

In [14]:
# %matplotlib inline
import os
import shutil
from collections import OrderedDict

import matplotlib
# matplotlib.use('WebAgg')
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from scipy.ndimage.measurements import label



from ssd import build_ssd
from data import detection_collate, GTDBDetection, BaseTransform, GTDBAnnotationTransform
import config
from torch.utils.data import Dataset, DataLoader
from utils import save_boxes, draw_box, _img_to_tensor, FixImgCoordinates, DrawAllBoxes, convert_to_binary, read_math, voting_equal, convert_to_binary, voting_algo

In [15]:
class ArgStub():

    def __init__ (self):
        self.cuda = False
        self.kernel = (1, 5)
        self.padding = (0, 2)
        self.phase = 'test'
        self.visual_threshold = 0.25
        self.verbose = False
        self.exp_name = 'SSD'
        self.model_type = 512
        self.use_char_info = False
        self.limit = -1
        self.cfg = 'hboxes512'
        self.batch_size = 8
        self.num_workers = 4
        self.neg_mining = True
        self.log_dir = 'logs'
        self.stride = 0.1
        self.window = 1200
        self.test_data = "testing_data"
        self.data_file = "train_data"
        self.dataset_root = "."
        self.save_folder = "./eval"
        self.exp_name = "testing"
        self.output_dir = "./eval/stitched_real_world_iter1"
        self.math_dir = "./eval/testing/"
        self.home_images = "./images/"
        self.math_ext = ".csv"
        self.algo_threshold = 30

In [16]:
class MathDetector():

    def __init__(self, weight_path, args):
        self.args = args
        net = build_ssd(args, 'test', config.exp_cfg[args.cfg], -1, args.model_type, 2)
        self._net = net # nn.DataParallel(net)
        weights = torch.load(weight_path, map_location = torch.device('cpu'))

        new_weights = OrderedDict()
        for k, v in weights.items():
            name = k[7:] # remove `module.`
            new_weights[name] = v

        self._net.load_state_dict(new_weights)
        self._net.eval()

        self.dataset = GTDBDetection(args, self.args.test_data, split='test',
                                transform=BaseTransform(self.args.model_type, (246, 246, 246)),
                                target_transform=GTDBAnnotationTransform())

        self.data_loader = DataLoader(self.dataset, self.args.batch_size,
                                 num_workers=self.args.num_workers,
                                 shuffle=False, collate_fn=detection_collate,
                                 pin_memory=True)

        self.boxes = []
        self.scores = []


    def Detect (self, thres, images):

        done = 0

        for batch_idx, (images, targets, metadata) in enumerate(self.data_loader):
            done = done + len(images)

            with torch.no_grad():
                images = Variable(images)
                targets = [Variable(ann) for ann in targets]

            y, debug_boxes, debug_scores = self._net(images)  # forward pass
            detections = y.data

            k = 0
            for img, meta in zip(images, metadata):

                img_id = meta[0]
                x_l = meta[1]
                y_l = meta[2]

                img = img.permute(1, 2, 0)
                # scale each detection back up to the image
                scale = torch.Tensor([img.shape[1], img.shape[0],
                                      img.shape[1], img.shape[0]])

                recognized_boxes = []
                recognized_scores = []

                # [1,2,200,5]
                # we only care about math class
                # hence select detections[image_id, class, detection_id, detection_score]
                # class=1 for math
                i = 1
                j = 0

                while j < detections.size(2) and detections[k, i, j, 0] >= thres:  # TODO it was 0.6

                    score = detections[k, i, j, 0]
                    pt = (detections[k, i, j, 1:] * self.args.window).cpu().numpy()
                    coords = (pt[0] + x_l, pt[1] + y_l, pt[2] + x_l, pt[3] + y_l)
                    # coords = (pt[0], pt[1], pt[2], pt[3])
                    recognized_boxes.append(coords)
                    recognized_scores.append(score.cpu().numpy())

                    j += 1
                    print(j)

                save_boxes(self.args, recognized_boxes, recognized_scores, img_id)
        self.boxes = recognized_boxes
        self.scores = recognized_scores


    def DetectAny (self, thres, image):
        t = _img_to_tensor(image).unsqueeze(0)
        # fix box coordinates to image pixel coordinates
        self.Detect(thres, t)
        # coor_boxes = FixImgCoordinates(image, self.boxes)
        # new_boxes = self.Voting(t, coor_boxes)
        # self.boxes = coor_boxes

        return self.boxes, self.scores

    def Voting(self):

        pdf_list = []
        pdf_names_file = open(self.args.data_file, 'r')

        if os.path.exists(self.args.output_dir):
            shutil.rmtree(self.args.output_dir)

        for pdf_name in pdf_names_file:
            pdf_name = pdf_name.strip()

            if pdf_name != '':
                pdf_list.append(pdf_name)

        math_regions = {}

        for pdf_name in pdf_list:
            math_regions[pdf_name] = read_math(self.args, pdf_name)

        voting_ip_list = []
        for pdf_name in pdf_list:

            pages = np.unique(math_regions[pdf_name][:, 0])

            #args, math_regions, pdf_name, page_num
            for page_num in pages:
                current_math = math_regions[pdf_name][math_regions[pdf_name][:,0] == page_num]
                voting_ip_list.append([self.args, np.delete(current_math, 0, 1), pdf_name, page_num])

        # pool = Pool(processes=args.num_workers)
        out = [voting_algo(ip) for ip in voting_ip_list]

        for ip, final_math in zip(voting_ip_list, out):

            try:
                pdf_name = ip[2]
                page_num = ip[3]

                if len(final_math) == 0:
                    continue

                col = np.array([int(page_num)] * len(final_math))
                final_math = np.concatenate((col[:, np.newaxis], final_math), axis=1)

                math_file_path = os.path.join(self.args.output_dir, pdf_name + '.csv')

                if not os.path.exists(os.path.dirname(math_file_path)):
                    os.makedirs(os.path.dirname(math_file_path))

                math_file = open(math_file_path, 'a')

                np.savetxt(math_file, final_math, fmt='%.2f', delimiter=',')
                math_file.close()
            except Exception as e:
                print("Exception while processing ", pdf_name, " ", page_num, " ", sys.exc_info(), e)

In [17]:
md = MathDetector('AMATH512_e1GTDB.pth', ArgStub())
a = cv2.imread('images/test/2.jpg', cv2.IMREAD_COLOR)

In [18]:
b, s = md.DetectAny(0.2, a)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [13]:
md.Voting()

Processing  test  >  0


NameError: ignored